## Automatic Number Plate Recognition System

License plate detection is identifying the part of the car that is predicted to be the number plate. Recognition is identifying the values that make up the license plate. License plate detection and recognition is the technology that uses computer vision to detect and recognize a license plate from an input image of a car. This technology applies in many areas. On roads, it is used to identify the cars that are breaking the traffic rules. In security, it is used to capture the license plates of the vehicles getting into and out of certain premises. In parking lots, it is used to capture the license plates of the cars being parked. The list of its applications goes on and on.

<img src= "https://github.com/Asikpalysik/Automatic-License-Plate-Detection/blob/main/Presentation/Notebook2.png?raw=true" width="100%" align="center"  hspace="5%" vspace="5%"/>

### Importing Required Libraries

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array


### Parsing xml data and converting into csv


In [3]:

path = glob('/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [4]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/Applications/Deepakkr/SummerIntership/numberp...,207,356,174,287
1,/Applications/Deepakkr/SummerIntership/numberp...,39,108,129,157
2,/Applications/Deepakkr/SummerIntership/numberp...,289,421,188,232
3,/Applications/Deepakkr/SummerIntership/numberp...,150,378,217,269
4,/Applications/Deepakkr/SummerIntership/numberp...,208,370,89,173


In [5]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/Applications/Deepakkr/SummerIntership/numberplaterecognition/images',filename_image)
    return filepath_image
getFilename(filename)

'/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N107.jpeg'

In [6]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N107.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N113.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N62.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N89.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N88.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N63.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N77.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N112.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N106.jpeg',
 '/Applications/Deepakkr/SummerIntership/numberplaterecognition/images/N110.jpeg']

### Verify the data

In [ ]:
file_path = image_path[1]#path of our image N113.jpeg
img = cv2.imread(file_path) #read the image
# xmin-39/ymin-129/xmax-108/ymax-157 
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='N113.jpeg with bounding box')
fig.add_shape(type='rect',x0=39, x1=108, y0=129, y1=157, xref='x', yref='y',line_color='red')

## Data Preprocessing 


### Read the data

In [8]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

### Split train and test set


In [9]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [10]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

## Model building InceptionResNetV2

In [ ]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

#model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [ ]:
# Compile model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 73,602,946 (280.77 MB)

 Non-trainable params: 60,544 (236.50 KB)

### Model training


In [25]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=180,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - loss: 0.0480 - val_loss: 0.0327
Epoch 2/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - loss: 0.0175 - val_loss: 0.0291
Epoch 3/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 0.0103 - val_loss: 0.0227
Epoch 4/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 0.0075 - val_loss: 0.0137
Epoch 5/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 0.0043 - val_loss: 0.0158
Epoch 6/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - loss: 0.0031 - val_loss: 0.0118
Epoch 7/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - loss: 0.0024 - val_loss: 0.0108
Epoch 8/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.0020 - val_loss: 0.0092
Epoch 9/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - loss: 0.0017 - val_loss: 0.0086
Epoch 10/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - loss: 0.0015 - val_loss: 0.0086
Epoch 11/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - loss: 0.0017 - val_loss: 0.0114
Epoch 12/180
18/18 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - loss: 0.0

### Model save

In [26]:
model.save('./object_detection.h5')

In [ ]:
model.save('object_detection.keras')

In [29]:
!tensorboard --logdir="./object_detection"

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


<img src= "https://github.com/deepakkr7975/Automatic_Number_Plate_recognition_ANPR/blob/main/tensorboard.png?raw=true" width="80%" align="center" hspace="5%" vspace="5%"/>


## Creating Pipeline For prediction 


### Make prediction

In [35]:
# Load model
model = tf.keras.models.load_model('object_detection.keras')
print('Model loaded Sucessfully')

Model loaded Sucessfully


/opt/homebrew/Caskroom/miniconda/base/envs/ANPR_env/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:802: UserWarning:

Skipping variable loading for optimizer 'rmsprop', because it has 496 variables whereas the saved optimizer has 990 variables. 



In [ ]:
path = 'TEST/images2.jpeg'
image = load_img(path)
image = np.array(image,dtype=np.uint8) #(0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)

Height of the image = 459
Width of the image = 700


In [157]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 13 - TEST Image')

In [158]:
image_arr_224.shape

(224, 224, 3)

In [159]:
test_arr=image_arr_224.reshape(1,224,224,3)
test_arr.shape

(1, 224, 224, 3)

#### Denormalizing the output

In [160]:
# Make predictions
coords = model.predict(test_arr)
coords

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


array([[0.27248198, 0.7675609 , 0.48323026, 0.6518772 ]], dtype=float32)

In [161]:
# Denormalize the values
denorm = np.array([w,w,h,h])
coords = coords * denorm
coords

array([[190.73738456, 537.29262948, 221.80269071, 299.21164602]])

#### Drawing Bounding Box

In [162]:
coords = coords.astype(np.int32)
coords

array([[190, 537, 221, 299]], dtype=int32)

In [163]:
# Draw bounding on top the image
xmin,xmax,ymin,ymax = coords[0]
pt1 =(int(xmin),int(ymin))
pt2 =(int(xmax),int(ymax))
print(pt1, pt2)

(190, 221) (537, 299)


In [164]:
cv2.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))

### Creating Pipeline

In [ ]:

path = 'TEST/images2.jpeg'
def object_detection(path):
    
    # Read image
    image = load_img(path) 
    image = np.array(image,dtype=np.uint8)
    image1 = load_img(path,target_size=(224,224))
    
    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)
    
    # Make predictions
    coords = model.predict(test_arr)
    
    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)
    
    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(int(xmin),int(ymin))
    pt2 =(int(xmax),int(ymax))
    
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

image, cods = object_detection(path)

fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
(190, 221) (537, 299)


## Optical character Recognition (OCR)

### Tesseract Ocr
#### Extracting Number plate text from Image

In [170]:
img = np.array(load_img(path))
xmin ,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Cropped image')

In [171]:
# extract text from image
text = pt.image_to_string(roi)
print(text)

HR26DK8337

